In [1]:
from utils import * 
from algorithms import *
from agents import *

In [2]:
numRounds, numPeriods, numSteps = 4, 4, 16
numBuyers, numSellers, numTokens = 4, 4, 4
gameTypes, seed = '1111', None
gameData = [gameTypes, numBuyers, numSellers, numTokens, numRounds, numPeriods, numSteps, seed]
disclosure = ['step', 'currentBid', 'currentAsk', 'currentBidIdx', 'currentAskIdx','buy', 'sell', 'price', 'sale',
              'currentBidIdx_0', 'currentBidIdx_1', 'currentBidIdx_2', 'currentBidIdx_3',
              'currentAskIdx_0', 'currentAskIdx_1', 'currentAskIdx_2', 'currentAskIdx_3',
              'bids_0', 'bids_1', 'bids_2', 'bids_3',
              'asks_0', 'asks_1', 'asks_2', 'asks_3']
depth = 1
buyers = [ZeroIntelligence(gameData, disclosure, index=0, buyer=1, reinforcer=0),
          ZeroIntelligence(gameData, disclosure, index=0, buyer=1, reinforcer=0),
          ZeroIntelligence(gameData, disclosure, index=0, buyer=1, reinforcer=0),
          ZeroIntelligence(gameData, disclosure, index=0, buyer=1, reinforcer=0)]
sellers = [ZeroIntelligence(gameData, disclosure, index=0, buyer=0, reinforcer=0),
           ZeroIntelligence(gameData, disclosure, index=0, buyer=0, reinforcer=0),
           ZeroIntelligence(gameData, disclosure, index=0, buyer=0, reinforcer=0),
           ZeroIntelligence(gameData, disclosure, index=0, buyer=0, reinforcer=0)]
log = Log(gameData, disclosure,depth)
rnd = 0
state_record = []
reward_record = []

In [3]:
for rnd in range(numRounds):
    startRounds(gameData, log, buyers, sellers, rnd)
    #print(log.roundData.buyerValues.item())
    #print(log.roundData.sellerCosts.item())
    for period in range(numPeriods):
        startPeriods(buyers, sellers)
        for step in range(numSteps):
            startSteps(buyers, sellers)
            #buyers[0].state = generateState(buyers[0], disclosure, depth)
            bids, asks = collectOffers(buyers, sellers)
            currentAsk, currentAskIdx, currentBid, currentBidIdx = bestOffers(bids, asks)
            price, buy, sell = trade(buyers, sellers, currentAsk, currentAskIdx, currentBid, currentBidIdx)
            bprofit, sprofit = 0, 0
            if price > 0:
                buyers[currentBidIdx].transact(price)
                sellers[currentAskIdx].transact(price)
                bprofit = buyers[currentBidIdx].stepProfits
                sprofit = sellers[currentAskIdx].stepProfits
            reward_record.append(buyers[0].stepProfits)
            log.addStep([rnd, period, step, currentBid, currentAsk, currentBidIdx, currentAskIdx, buy, sell, price, price>0,
                         currentBidIdx==0, currentBidIdx==1, currentBidIdx==2, currentBidIdx==3, 
                         currentAskIdx==0, currentAskIdx==1, currentAskIdx==2, currentAskIdx==3, 
                         bids[0], bids[1], bids[2], bids[3], 
                         asks[0], asks[1], asks[2], asks[3], 
                         bprofit, sprofit])
            observe(buyers, sellers, log.disclose())
            agentsObserve(buyers, sellers)
            agentsTrain(buyers, sellers)
            endSteps(buyers, sellers)
            state_record.append([step, buyers[0].periodTrades, bids[0], bids[1], bids[2], bids[3], asks[0], asks[1], asks[2], asks[3], currentBid, currentBidIdx, currentAsk, currentAskIdx, buy, sell, price, price>0])
        endPeriods(buyers, sellers)
    endRounds(buyers, sellers)

Empty DataFrame
Columns: [rnd, period, step, currentBid, currentAsk, currentBidIdx, currentAskIdx, buy, sell, price, sale, currentBidIdx_0, currentBidIdx_1, currentBidIdx_2, currentBidIdx_3, currentAskIdx_0, currentAskIdx_1, currentAskIdx_2, currentAskIdx_3, bids_0, bids_1, bids_2, bids_3, asks_0, asks_1, asks_2, asks_3, bprofit, sprofit]
Index: []

[0 rows x 29 columns]
[0, 0, 0, 68.5, 47.0, 1, 1, True, True, 57.75, True, False, True, False, False, False, True, False, False, 65.3, 68.5, 57.2, 50.0, 60.4, 47.0, 54.8, 47.9, 23.254796328092468, 21.129994459436176]


ValueError: operands could not be broadcast together with shapes (25,) (23,) 

In [ ]:
np.array(state_record)

# Compute Mean 

In [25]:
for i in state_record:
    print(i)

[0, 0, 72.8, 73.7, 60.6, 59.2, 53.2, 53.7, 59.8, 53.3, 73.7, 1, 53.2, 0, True, True, 63.45, True]
[1, 0, 61.1, 43.4, 48.7, 74.2, 56.9, 51.4, 63.5, 65.6, 74.2, 3, 51.4, 1, True, True, 62.8, True]
[2, 0, 53.4, 70.3, 49.5, 77.6, 61.9, 66.8, 59.8, 56.5, 77.6, 3, 56.5, 3, True, True, 67.05, True]
[3, 0, 47.4, 51.7, 54.0, 76.8, 49.8, 70.2, 48.9, 75.6, 76.8, 3, 48.9, 2, True, True, 62.849999999999994, True]
[4, 0, 71.6, 76.7, 66.8, 55.9, 50.8, 70.8, 63.2, 64.7, 76.7, 1, 50.8, 0, True, True, 63.75, True]
[5, 1, 71.3, 58.1, 65.7, 51.0, 73.6, 52.0, 53.1, 74.1, 71.3, 0, 52.0, 1, True, True, 61.65, True]
[6, 1, 63.7, 54.3, 70.5, 43.8, 66.9, 61.5, 58.9, 72.4, 70.5, 2, 58.9, 2, True, True, 64.7, True]
[7, 1, 52.5, 58.9, 40.5, 66.4, 53.8, 66.9, 59.0, 67.6, 66.4, 3, 53.8, 0, True, True, 60.1, True]
[8, 1, 55.8, 57.7, 68.5, nan, 65.7, 61.0, 68.1, 61.9, 68.5, 2, 61.0, 1, True, True, 64.75, True]
[9, 1, 48.3, 66.3, 40.0, nan, 74.6, 80.0, 53.0, 69.9, 66.3, 1, 53.0, 2, True, True, 59.65, True]
[10, 1, 67.1

In [26]:
states = np.array(state_record)
cols = ['step', 'sales', 'bids1','bids2','bids3','bids4','asks1','asks2','asks3','asks4', 'currentBid', 'currentBidIdx','currentAsk', 'currentAskIdx', 'buy', 'sell', 'price', 'sale']
mean_df = pd.DataFrame(np.mean(states,axis=0).reshape(1,-1), columns = cols)
mean_df.to_csv('mean_state.csv')
cols = ['step', 'sales', 'bids1','bids2','bids3','bids4', 'asks1','asks2','asks3','asks4', 'currentBid', 'currentBidIdx','currentAsk', 'currentAskIdx', 'buy', 'sell', 'price', 'sale']
std_df = pd.DataFrame(np.std(states,axis=0).reshape(1,-1), columns = cols)
std_df.to_csv('std_state.csv')

In [30]:
log.stepData

,rnd,period,step,currentBid,currentAsk,buy,sell,price,sale,currentBidIdx_0,...,bids_0,bids_1,bids_2,bids_3,asks_0,asks_1,asks_2,asks_3,bprofit,sprofit
0,0,0,0,73.7,53.2,True,True,63.45,True,False,...,72.8,73.7,60.6,59.2,53.2,53.7,59.8,53.3,21.358619,19.771800
1,0,0,1,74.2,51.4,True,True,62.80,True,False,...,61.1,43.4,48.7,74.2,56.9,51.4,63.5,65.6,20.886139,13.660562
2,0,0,2,77.6,56.5,True,True,67.05,True,False,...,53.4,70.3,49.5,77.6,61.9,66.8,59.8,56.5,15.763783,19.189132
3,0,0,3,76.8,48.9,True,True,62.85,True,False,...,47.4,51.7,54.0,76.8,49.8,70.2,48.9,75.6,19.805781,17.619281
4,0,0,4,76.7,50.8,True,True,63.75,True,False,...,71.6,76.7,66.8,55.9,50.8,70.8,63.2,64.7,13.117742,16.252150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,3,3,11,50.4,63.9,True,False,63.90,True,False,...,NaN,50.4,48.3,NaN,63.9,76.3,NaN,71.2,9.891025,9.049033
252,3,3,12,49.7,63.1,True,False,63.10,True,False,...,NaN,39.8,49.7,NaN,NaN,63.1,NaN,84.3,19.971174,9.314967
253,3,3,13,58.6,62.5,True,True,60.55,True,False,...,NaN,58.6,51.9,NaN,NaN,72.2,NaN,62.5,10.768676,3.075911
254,3,3,14,76.4,74.8,True,True,75.60,True,False,...,NaN,NaN,76.4,NaN,NaN,74.8,NaN,NaN,5.568045,20.398998


In [27]:
log.stepData.tail(1000)[['currentBidIdx', 'bprofit']].groupby('currentBidIdx').sum()/log.stepData.tail(1000).period.nunique()

KeyError: "['currentBidIdx'] not in index"

In [ ]:
log.stepData.tail(1000)[['currentAskIdx', 'bprofit']].groupby('currentAskIdx').sum()/log.stepData.tail(1000).period.nunique()

In [ ]:
log.getPeriod(0, period-1)

In [ ]:
log.graphLearning(rolling_window = 500, trackBuyersIdx=[0], trackSellersIdx=[0])

In [ ]:
log.graphSales(0, period-1, trackBuyersIdx=[0], trackSellersIdx=[0])

In [ ]:
log.stepData.tail(15)

In [ ]:
log.getPeriod(0, period-1).sum()

In [ ]:
log.graphOffers(0, period-1, trackBuyersIdx=[0], trackSellersIdx=[0])

In [ ]:
log.graphTraining(maxNumPeriods=period-1, skip = int((period-1)/20), trackBuyersIdx = [0])